In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
df = pd.read_csv('/content/chatbot_intent_classification.csv')

In [5]:
df[:30]

,user_input,intent
0,"Cancel my subscription, please.",cancellation
1,I forgot my password.,password_reset
2,Can you help me with my account?,account_help
3,I want to return my order.,return_request
4,What time do you open?,business_hours
5,I forgot my password.,password_reset
6,Where is my package?,order_status
7,I forgot my password.,password_reset
8,"Cancel my subscription, please.",cancellation
9,"Cancel my subscription, please.",cancellation


In [6]:
few_shot_examples = [
    {"user_input": "Cancel my subscription, please.", "intent": "cancellation"},
    {"user_input": "I forgot my password.", "intent": "password_reset"},
    {"user_input": "What time do you open?", "intent": "business_hours"},
    {"user_input": "Where is my package?", "intent":	"order_status"},
    {"user_input": "I want to return my order.", "intent":	"return_request"},
    {"user_input":"Can you help me with my account?","intent": "account_help"},
    {"user_input":"	How can I update my payment method?","intent": "payment_update"},
    {"user_input":"	I need technical support.","intent": "technical_support"},

]

In [7]:
def create_few_shot_prompt(user_input, examples, instruction="Classify the intent of the following user input:"):
    """
    Builds a prompt by providing a few examples followed by the new input.
    """
    prompt = "Below are a few examples of user inputs and their intents:\n"
    for ex in examples:
        prompt += f"User: \"{ex['user_input']}\"\nIntent: {ex['intent']}\n\n"
    prompt += f"{instruction}\nUser: \"{user_input}\"\nIntent:"
    return prompt


In [8]:
model_name = "google/flan-t5-large"  # Recommended few-shot model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def few_shot_predict(user_input, examples=few_shot_examples):
    """
    Generates a prediction using few-shot prompting.
    """
    prompt = create_few_shot_prompt(user_input, examples)
    input_ids = tokenizer.encode(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(input_ids, max_length=50)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction.strip()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
results = []
for idx, row in df.iterrows():  # Evaluate on first 10 samples for demonstration
    input_text = row["user_input"]
    true_intent = row["intent"]
    pred_intent = few_shot_predict(input_text)
    results.append({"user_input": input_text, "true_intent": true_intent, "predicted_intent": pred_intent})
    print(f"Input: {input_text}")
    print(f"True Intent: {true_intent}")
    print(f"Predicted Intent: {pred_intent}")
    print("-" * 50)

# ----------------------------
# Step 5: Compute a Simple Accuracy on the Sample
# ----------------------------
true_labels = [r["true_intent"] for r in results]
pred_labels = [r["predicted_intent"] for r in results]
accuracy = np.mean([t == p for t, p in zip(true_labels, pred_labels)])
print("Accuracy on sample:", accuracy)

Input: Cancel my subscription, please.
True Intent: cancellation
Predicted Intent: cancellation
--------------------------------------------------
Input: I forgot my password.
True Intent: password_reset
Predicted Intent: password_reset
--------------------------------------------------
Input: Can you help me with my account?
True Intent: account_help
Predicted Intent: account_help
--------------------------------------------------
Input: I want to return my order.
True Intent: return_request
Predicted Intent: return_request
--------------------------------------------------
Input: What time do you open?
True Intent: business_hours
Predicted Intent: business_hours
--------------------------------------------------
Input: I forgot my password.
True Intent: password_reset
Predicted Intent: password_reset
--------------------------------------------------
Input: Where is my package?
True Intent: order_status
Predicted Intent: order_status
--------------------------------------------------